In [2]:
# Setup the Jupyter version of Dash
##from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash import Dash, dcc, html, callback_context
# Configure the necessary Python module imports
import dash_leaflet as dl
import plotly.express as px
from dash import dash_table



# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
#from animal_shelter import AnimalShelter
from mongodb_crud import MongoDBCRUD


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32218
database_name = "AAC"
collection_name = "animals"

mongo_crud = MongoDBCRUD(username, password, host, port, database_name)
#shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

#df = pd.DataFrame.from_records(mongo_crud.read({}))



# Read data from MongoDB and create a DataFrame
documents = mongo_crud.read(collection_name, {})  # Assuming read() is correctly implemented
if isinstance(documents, list):
    df = pd.DataFrame(documents)
    df.drop(columns=['_id'], inplace=True, errors='ignore')
else:
    print("Error fetching data:", documents)
    df = pd.DataFrame()  # Create an empty DataFrame in case of error

    

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Janncy Mota'))),
    html.Hr(),
    html.Div(
        className='buttonRow',
        style={'display': 'flex'},
        children=[
            html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
            html.Button(id='submit-button-two', n_clicks=0, children='Dogs'),
        ]
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6')
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it


@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks')],
)
def on_click(btn1_clicks, btn2_clicks):
    # Default query to fetch all records if none of the buttons were clicked
    query = {}
    
    # Determine which button was clicked last based on the click counts
    if btn1_clicks > btn2_clicks:
        query = {"animal_type": "Cat"}
    elif btn2_clicks > btn1_clicks:
        query = {"animal_type": "Dog"}
    
    # Fetch and filter data based on the query
    documents = mongo_crud.read(collection_name, query)
    if isinstance(documents, list):
        df_filtered = pd.DataFrame(documents)
        df_filtered.drop(columns=['_id'], inplace=True, errors='ignore')
        return df_filtered.to_dict('records')
    
    return [] 

                                    
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData) if viewData else pd.DataFrame()
    
    if dff.empty or index is None or len(index) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip("No data available"),
                dl.Popup("No selection or data is empty")
            ])
        ])
#FIXME Add in the code for your geolocation chart


    row = index[0] if len(dff) > index[0] else 0
    
    lat, lon = (30.75, -97.48)  # Default coordinates
    if "latitude" in dff.columns and "longitude" in dff.columns:
        lat = dff.loc[row, "latitude"]
        lon = dff.loc[row, "longitude"]

    breed = dff.loc[row, "breed"] if "breed" in dff.columns else "Breed info not available"
    name = dff.loc[row, "name"] if "name" in dff.columns else "Name info not available"
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1(name),
                    html.P(breed)
                ])
            ])
        ])
    ]


    
app.run_server(debug=True, port=8000)

Error fetching data: nv-desktop-services.apporto.com:32218: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 65d98fa2bddf498027b2a74b, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 32218) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:32218: [Errno 111] Connection refused')>]>


TypeError: an integer is required (got type NoneType)